# Population Data from CSV

This notebooks reads sample population data from `data/atlantis.csv` and plots it using Matplotlib. Edit `data/atlantis.csv` and re-run this cell to see how the plots change!

In [ ]:
pip install "ray[rllib]" tensorflow

In [ ]:
pip install "ray[rllib]" torch

In [ ]:
pip install swig

In [ ]:
pip install gymnasium[box2d]

In [ ]:
!pip install wandb

In [ ]:
from ray.rllib.algorithms.dreamerv3 import DreamerV3Config
config = DreamerV3Config()
algo = config.build(env="CartPole-v1")  # doctest: +SKIP
algo.train()  # doctest: +SKIP

In [ ]:
from ray import tune

# Configure.
from ray.rllib.algorithms.ppo import PPOConfig
config = PPOConfig().environment(env="CartPole-v1").training(train_batch_size=4000)

# Train via Ray Tune.
tune.run("PPO", config=config)

In [ ]:
# Configure.
from ray.rllib.algorithms.ppo import PPOConfig
config = PPOConfig().environment(env="CartPole-v1").training(train_batch_size=4000)

# Build.
algo = config.build()

# Train.
while True:
    print(algo.train())

In [ ]:
from ray import tune
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config


def _env_creator(ctx):
    import flappy_bird_gymnasium  # doctest: +SKIP
    import gymnasium as gym
    from supersuit.generic_wrappers import resize_v1
    from ray.rllib.algorithms.dreamerv3.utils.env_runner import NormalizedImageEnv

    return NormalizedImageEnv(
        resize_v1(  # resize to 64x64 and normalize images
            gym.make("FlappyBird-rgb-v0", audio_on=False), x_size=64, y_size=64
        )
    )


# Register the FlappyBird-rgb-v0 env including necessary wrappers via the
# `tune.register_env()` API.
tune.register_env("flappy-bird", _env_creator)

# Define the `config` variable to use for training.
config = (
    DreamerV3Config()
    # set the env to the pre-registered string
    .environment("flappy-bird")
    # play around with the insanely high number of hyperparameters for DreamerV3 ;) 
    .training(
        model_size="S",
        training_ratio=1024,
    )
)
#dasgdashadhadh
# Run the tuner job.
results = tune.Tuner(trainable="DreamerV3", param_space=config).fit()

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import random
import wandb

#мы заменили float и np.array
class GoLeftEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left.
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  FIRST = 0
  SECOND = 1
  THIRD = 2
  FORTH = 3
  FIFTH= 4
  SIXTH = 5
  SEVENTH = 6

  def __init__(self, grid_size=10, render_mode="console"):
    super(GoLeftEnv, self).__init__()
    self.render_mode = render_mode

    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]]).flatten('C')

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 7
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.Box(low=-16, high=16,
                                        shape=(68,), dtype=np.float32)
  #попробовать переделать всё в np.array

  def reset(self, seed=None, options=None):
    """
    Important: the observation must be a numpy array
    :return: (np.array)
    """
    super().reset(seed=seed, options=options)


    # Initialize the agent at the right of the grid
    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]])



    # here we convert to float32 to make it more general (in case we want to use continuous actions)
    return self.fields.astype(np.float32).flatten('C'), {}

  def step(self, action):

     self.fields = self.fields.reshape(17, 4)

     def first():
            numberforcycles = 15

            balcycle1 = 0

            roll = self.fields[16][0]

            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextroll = random.choices(lst, weights=weights, k=1)
            nextroll = nextroll[0]
            self.fields[16][0] = nextroll
            #print("Выпала" , roll)
            arrplayer1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
          #расположение фишек игрока 1
              if ballnow != 0:
               arrplayer1.append(ballindex)
              balcycle1 += 1
            if self.fields[8][3] == 1:
                arrplayer1.append(8)
            #print(arrplayer1)

            balcycle1 = 0
            arrchoice1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
              if ballnow != 0:
                ballfuture = ballindex + roll
                if ballfuture <= numberforcycles:
                 if ballfuture not in arrplayer1:
                  arrchoice1.append(ballfuture)
              # print("Выпала" , self.roll)
                  #print("текущая позиция" , ballindex)
                  #print("возможная позиция" , ballfuture)
              balcycle1 += 1



            if arrchoice1:

              #случайный бот:
              playeronemove = random.choice(arrchoice1)

              #Очень жадный бот:
              #playeronemove = len(arrchoice1)
              #playeronemove = arrchoice1[playeronemove-1]

              #Жадный бот:
              #playeronemove = 33
              #for arrchoice in arrchoice1:
              #  if self.fields[arrchoice][1] == 1 or (self.fields[arrchoice][3] == 1 and self.fields[arrchoice][0] >= 5 and self.fields[arrchoice][0] <= 12):
              #    playeronemove = self.fields[arrchoice][0]
              #if playeronemove == 33:
              #  playeronemove = len(arrchoice1)
              #  playeronemove = arrchoice1[playeronemove-1]

              #playeronemove = random.choice(arrchoice1)
              playeronepos = playeronemove - roll
              #print(playeronepos)
              self.fields[playeronepos][2] = self.fields[playeronepos][2] - 1
              self.fields[playeronemove][2] = self.fields[playeronemove][2] + 1
              if self.fields[playeronemove][3] == 1 and playeronemove >= 5 and playeronemove <= 12:
                self.fields[playeronemove][3] = 0
                self.fields[0][3] = self.fields[0][3] + 1
                #print("мы забрали шашку на" , self.fields[playeronemove][0])
              #print("боту выпал", roll)
              #print("бот ходит", playeronemove)
              return playeronemove

     def second():
            numberforcycles = 15

            agentroll = self.fields[16][1]


            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextagentroll = random.choices(lst, weights=weights, k=1)
            nextagentroll = nextagentroll[0]
            self.fields[16][1] = nextagentroll
            #print("Агенту Выпал" , agentroll)

            agentcycle = 0
            arrplayeragent = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #расположение фишек игрока 2
              if agentballnow != 0:
               arrplayeragent.append(agentballindex)
              agentcycle += 1
            if self.fields[8][2] == 1:
                arrplayeragent.append(8)

            #print("фишки агента", arrplayeragent)

            agentcycle = 0
            arrplayeragent2 = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #будущее фишек игрока 2
              if agentballnow != 0:
               agentballfuture = agentballindex + agentroll
               if agentballfuture <= numberforcycles:
                if agentballfuture not in arrplayeragent:
                  arrplayeragent2.append(agentballfuture)
              agentcycle += 1

            #print(arrplayeragent2)



            if arrplayeragent2:
              playeragentmove = len(arrplayeragent2)
              playeragentmove = arrplayeragent2[0]#arrplayeragent2[playeragentmove-1]

              if action == self.FIRST:
                if len(arrplayeragent2) > 0:
                 playeragentmove = arrplayeragent2[0]
                else:
                 playeragentmove = playeragentmove
              elif action == self.SECOND:
               if len(arrplayeragent2) > 1:
                playeragentmove = arrplayeragent2[1]
               else:
                playeragentmove = playeragentmove
              elif action == self.THIRD:
               if len(arrplayeragent2) > 2:
                playeragentmove = arrplayeragent2[2]
               else:
                playeragentmove = playeragentmove
              elif action == self.FORTH:
               if len(arrplayeragent2) > 3:
                playeragentmove = arrplayeragent2[3]
               else:
                playeragentmove = playeragentmove
              elif action == self.FIFTH:
               if len(arrplayeragent2) > 4:
                playeragentmove = arrplayeragent2[4]
               else:
                playeragentmove = playeragentmove
              elif action == self.SIXTH:
               if len(arrplayeragent2) > 5:
                playeragentmove = arrplayeragent2[5]
               else:
                playeragentmove = playeragentmove
              elif action == self.SEVENTH:
               if len(arrplayeragent2) > 6:
                playeragentmove = arrplayeragent2[6]
               else:
                playeragentmove = playeragentmove
              else:
                raise ValueError("Received invalid action={} which is not part of the action space".format(action))



              #print("Агент делает выбор", playeragentmove)

              #playeragentmove = random.choice(arrplayeragent)
              playeragentpos = playeragentmove - agentroll
              self.fields[playeragentpos][3] = self.fields[playeragentpos][3] - 1
              self.fields[playeragentmove][3] = self.fields[playeragentmove][3] + 1
              if self.fields[playeragentmove][2] == 1 and playeragentmove >= 5 and playeragentmove <= 12:
                self.fields[playeragentmove][2] = 0
                self.fields[0][2] = self.fields[0][2] + 1
                #print("Агент забрал шашку на" , self.fields[playeragentmove][0])
              #print("Агенту выпал" , agentroll)
              #print("Агент ходит" , playeragentmove)
              return playeragentmove
            else:
              rrr = 0#print("Выпал 0 и агент не делает выбора")
     #while True:
     # thisbotmove = first()
     # if thisbotmove != 4 or thisbotmove != 8 or thisbotmove != 14:
     #    #print("нет второго хода бота")
     #    break

     if self.fields[16][3] != 1:
      thisbotmove = first()
      #print(first())
      if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
       #print("Второй ход бота", thisbotmove)
       self.fields[16][2] = 1
      else:
       self.fields[16][2] = 0
    # if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
     if self.fields[16][2] != 1:
      thisagentmove = second()
      #print(second())
      if thisagentmove == 4 or thisagentmove == 8 or thisagentmove == 14:
        #print("Второй ход агента", thisagentmove)
        self.fields[16][3] = 1
      else:
        self.fields[16][3] = 0
     #while True:
     # thisagentmove =  second()
     # if thisagentmove != 4 or thisagentmove != 8 or thisagentmove != 14:
     #    #print("нет второго хода агента")
     #    break


     terminated = bool(self.fields[15][2] >= 7 or self.fields[15][3] >= 7)
     truncated = False  # we do not limit the number of steps here

     reward = 0
     if self.fields[15][3] >= 7 and self.fields[15][2] != 7:
       reward = 1

    # Optionally we can pass additional info, we are not using that for now
     info = {}

     return np.array(self.fields).astype(np.float32).flatten('C'), reward, terminated, truncated, info


  def render(self):
        # agent is represented as a cross, rest as a dot
        if self.render_mode == "console":
            1==1

    #print(self.fields[0])

  def close(self):
    pass

In [ ]:
from ray.rllib.algorithms.alpha_zero import AlphaZeroConfig
config = AlphaZeroConfig()   
config = config.training(sgd_minibatch_size=256)   
config = config.resources(num_gpus=0)   
config = config.rollouts(num_rollout_workers=2)   
print(config.to_dict()) 
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="CartPole-v1")  
algo.train() 

In [ ]:
from ray import tune

# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
tune.register_env("my_env", lambda config: GoLeftEnv())

In [ ]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces


class GoLeftEnv(gym.Env):
    """
    Custom Environment that follows gym interface.
    This is a simple env where the agent must learn to go always left.
    """

    # Because of google colab, we cannot implement the GUI ('human' render mode)
    metadata = {"render_modes": ["console"]}

    # Define constants for clearer code
    LEFT = 0
    RIGHT = 1

    def __init__(self, grid_size=10, render_mode="console"):
        super(GoLeftEnv, self).__init__()
        self.render_mode = render_mode

        # Size of the 1D-grid
        self.grid_size = grid_size
        # Initialize the agent at the right of the grid
        self.agent_pos = grid_size - 1

        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions, we have two: left and right
        n_actions = 2
        self.action_space = spaces.Discrete(n_actions)
        # The observation will be the coordinate of the agent
        # this can be described both by Discrete and Box space
        self.observation_space = spaces.Box(
            low=0, high=self.grid_size, shape=(1,), dtype=np.float32
        )

    def reset(self, seed=None, options=None):
        """
        Important: the observation must be a numpy array
        :return: (np.array)
        """
        super().reset(seed=seed, options=options)
        # Initialize the agent at the right of the grid
        self.agent_pos = self.grid_size - 1
        # here we convert to float32 to make it more general (in case we want to use continuous actions)
        return np.array([self.agent_pos]).astype(np.float32), {}  # empty info dict

    def step(self, action):
        if action == self.LEFT:
            self.agent_pos -= 1
        elif action == self.RIGHT:
            self.agent_pos += 1
        else:
            raise ValueError(
                f"Received invalid action={action} which is not part of the action space"
            )

        # Account for the boundaries of the grid
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

        # Are we at the left of the grid?
        terminated = bool(self.agent_pos == 0)
        truncated = False  # we do not limit the number of steps here

        # Null reward everywhere except when reaching the goal (left of the grid)
        reward = 1 if self.agent_pos == 0 else 0

        # Optionally we can pass additional info, we are not using that for now
        info = {}

        return (
            np.array([self.agent_pos]).astype(np.float32),
            reward,
            terminated,
            truncated,
            info,
        )

    def render(self):
        # agent is represented as a cross, rest as a dot
        if self.render_mode == "console":
            print("." * self.agent_pos, end="")
            print("x", end="")
            print("." * (self.grid_size - self.agent_pos))

    def close(self):
        pass

In [ ]:
from ray.rllib.algorithms.alpha_zero import AlphaZeroConfig
from ray import air
from ray import tune
config = AlphaZeroConfig()
# Print out some default values.
print(config.shuffle_sequences) 
# Update the config object.
config.training(lr=tune.grid_search([0.001, 0.0001]))  
# Set the config object's env.
config.environment(env="CartPole-v1")   
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner( 
    "AlphaZero",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit() 

In [ ]:
from ray.rllib.algorithms.apex_dqn.apex_dqn import ApexDQNConfig
config = ApexDQNConfig()
print(config.replay_buffer_config) 
replay_config = config.replay_buffer_config.update( 
    {
        "capacity": 100000,
        "prioritized_replay_alpha": 0.45,
        "prioritized_replay_beta": 0.55,
        "prioritized_replay_eps": 3e-6,
    }
)
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=4)  
config = config.environment("CartPole-v1")  
algo = config.build() 
for _ in range(5):
    print(algo.train())  # 3. train it,

In [ ]:
from ray.rllib.algorithms.appo import APPOConfig
config = APPOConfig().training(lr=0.01, grad_clip=30.0)
config = config.resources(num_gpus=0)
config = config.rollouts(num_rollout_workers=2)
config = config.environment("my_env")
print(config.to_dict())  
# Build an Algorithm object from the config and run 1 training iteration.
algo = config.build()  

for _ in range(5):
    print(algo.train())  # 3. train it,

In [ ]:
         from ray.rllib.algorithms.dreamer import DreamerConfig
         config = DreamerConfig().training(gamma=0.9, lr=0.01)  # doctest: +SKIP
         config = config.resources(num_gpus=0)  # doctest: +SKIP
         config = config.rollouts(num_rollout_workers=0)  # doctest: +SKIP
         print(config.to_dict())  # doctest: +SKIP
         # Build a Algorithm object from the config and run 1 training iteration.
         algo = config.build(env="Taxi-v3")  # doctest: +SKIP
         algo.train()  # doctest: +SKIP

In [ ]:
         from ray.rllib.algorithms.dreamerv3 import DreamerV3Config
         config = DreamerV3Config()
         config = config.training(  # doctest: +SKIP
             batch_size_B=8, model_size="M"
         )
         config = config.resources(num_learner_workers=4)  # doctest: +SKIP
         print(config.to_dict())  # doctest: +SKIP
         # Build a Algorithm object from the config and run 1 training iteration.
         algo = config.build(env="CartPole-v1")  # doctest: +SKIP
         algo.train()  # doctest: +SKIP


In [ ]:
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config

# Run with:
# python run_regression_tests.py --dir [this file]

config = (
    DreamerV3Config()
    .environment("CartPole-v1")
    .training(
        model_size="XS",
        training_ratio=1024,
    )
)

tune.Tuner(trainable="DreamerV3", param_space=config).fit()


In [ ]:
from ray import air
from ray import tune
from ray.rllib.algorithms.dreamer import DreamerConfig
config = DreamerConfig()
# Print out some default values.
# Update the config object.
config = config.training(  
    lr=tune.grid_search([0.001, 0.0001]))
# Set the config object's env.
config = config.environment(env="CartPole-v1")  
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.Tuner(  
    "Dreamer",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()

In [ ]:
from ray.rllib.algorithms.dreamer import DreamerConfig
config = DreamerConfig().training(gamma=0.9, lr=0.01)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=0)  
print(config.to_dict())  
# Build a Algorithm object from the config and run 1 training iteration.
algo = config.build(env="CartPole-v1")  
algo.train()  

In [ ]:
from ray.rllib.algorithms.dreamer import DreamerConfig
from ray.tune.logger import pretty_print


algo = (
    DreamerConfig()
    .rollouts(num_rollout_workers=1)
    .resources(num_gpus=0)
    .environment(env="CartPole-v1")
    .build()
)

for i in range(10):
    result = algo.train()
    print(pretty_print(result))

    if i % 5 == 0:
        checkpoint_dir = algo.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

In [ ]:
env = GoLeftEnv()

for _ in range(100):
    obs = env.reset()
    while True:
        action = np.array(1)
        obs, r, done, _, _ = env.step(action)
        print(obs)
        if done:
            break

In [ ]:
import ray

ray.init()

In [ ]:
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config

config = (
    DreamerV3Config()
    # set the env to the pre-registered string
    .environment("my_env")
    # play around with the insanely high number of hyperparameters for DreamerV3 ;) 
    .training(
        model_size="S",
        training_ratio=1024,
    )
)

results = tune.Tuner(trainable="DreamerV3", param_space=config).fit()

print(777777)

In [ ]:
from ray import tune

# Configure.
from ray.rllib.algorithms.ppo import PPOConfig
config = PPOConfig().environment(env="my_env").training(train_batch_size=4000)

# Train via Ray Tune.
tune.run("PPO", config=config)

In [ ]:
# Configure.
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config
config = DreamerV3Config().environment(env="CartPole-v1")

# Build.
algo = config.build()

# Train.
while True:
    print(algo.train())

In [ ]:
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config

config = (  # 1. Configure the algorithm,
    DreamerV3Config()
    .environment("MountainCar-v0")
    .framework("tf2")
    .training(model_size="XS",training_ratio=1024,)
)

algo = config.build()  # 2. build the algorithm,

for _ in range(5):
    print(algo.train())  # 3. train it,

In [ ]:
from ray.rllib.algorithms.dreamerv3.dreamerv3 import DreamerV3Config

config = (  # 1. Configure the algorithm,
    DreamerV3Config()
    .environment("ALE/Alien-v5")
    .rollouts(num_rollout_workers=1)
)

algo = config.build()  # 2. build the algorithm,

for _ in range(5):
    print(algo.train())  # 3. train it,

In [ ]:
from ray import tune

tune.run("PPO",
         config={"env": "my_env",
                 "evaluation_interval": 10,
                 "evaluation_num_episodes": 50,
                 "num_workers": 1,
                 # other configurations go here, if none provided, then default configurations will be used
                 },
         storage_path="Urrnow4",
         checkpoint_freq=10
         )

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm

algo2 = Algorithm.from_checkpoint("/workspaces/codespaces-jupyter/notebooks/Urrnow2/PPO/PPO_my_env_8a337_00000_0_2023-08-19_11-35-47/checkpoint_000125")

In [ ]:
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<100:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
  print(episode_reward)